In [1]:
import pandas as pd 
import numpy as np
import requests
from shapely.geometry import CAP_STYLE, JOIN_STYLE
from geopandas import GeoSeries
from shapely.geometry import Point 
from pyproj import Proj
import geopandas as gpd

In [2]:
import pymysql
config = {
    'host': '192.168.1.31',
    'port': 4000,
    'user': 'liyufan',
    'passwd': 'liyufan@2020',
    'charset':'utf8mb4',
    'cursorclass':pymysql.cursors.DictCursor
    }
conn = pymysql.connect(**config)

In [3]:
store_close = pd.read_excel('0125/各省份低效清单1.25.xlsx')

In [4]:
store_close.head()

,区域,门店编码,countyId,mapX,mapY
0,湖北区域,420112101531,420112,114.127449,30.617956
1,湖北区域,420111103060,420111,114.318237,30.520592
2,湖北区域,411502100337,411502,114.092239,32.110161
3,湖北区域,420116102966,420116,114.351501,31.095409
4,湖北区域,411522100384,411522,114.916229,32.026531


In [5]:
store_close.dtypes

区域           object
门店编码          int64
店名           object
countyId      int64
mapX        float64
mapY        float64
dtype: object

In [121]:
temp_province_store.head(2)

,区域,门店编码,countyId,mapX,mapY
7317,福建区域,350302100524,350302,119.030449,25.304600
7323,福建区域,50190034,350112,119.511253,25.959873


In [6]:
for province in store_close['区域'].unique():
    print(province+"--------------")
#     if province != '福建区域':
#         continue
    province_list = []
    temp_province_store = store_close[store_close['区域'] == province].copy()
    temp_province_store['门店编码'] = temp_province_store['门店编码'].astype(str)
    #从数据库调门店地理位置
#     store_location = pd.read_sql("select storeCode,countyId, areaName, mapX, mapY from xsyx_frxs_base.t_store where storeCode in" +
#                                  str(tuple(temp_province_store['门店编码'].values.tolist())), conn)
    temp_store_with_location = temp_province_store.copy()
    
    #自定义projection  
    p=Proj('+proj=aea +lon_0='+str(temp_store_with_location['mapX'].mean())+' +lat_1='+ str(temp_store_with_location['mapY'].min()) 
            +' +lat_2='+ str(temp_store_with_location['mapY'].max()) +' +ellps=WGS84')
    #经纬度转成平面坐标系
    temp_store_with_location['flat_x'],temp_store_with_location['flat_y'] = zip(*temp_store_with_location.apply(lambda df: p(df['mapX'], df['mapY']), axis=1))        
    #将平面坐标转化为点
    #temp_store_with_location['flat_x'] = temp_store_with_location['flat_x'].astype(float)
    #temp_store_with_location['flat_y'] = temp_store_with_location['flat_y'].astype(float)
    temp_store_with_location['point_xy'] =temp_store_with_location.apply(lambda df:Point(df['flat_x'],df['flat_y']),axis=1) 
    #temp_store_with_gdf = gpd.GeoDataFrame(temp_store_with_location, geometry=gpd.points_from_xy(temp_store_with_location.flat_x, temp_store_with_location.flat_x))       
    #create a buffer for each point
    temp_store_with_location['buffer'] = temp_store_with_location.apply(lambda df:df['point_xy'].buffer(500, cap_style=1),axis=1)
        
    #print("------store_location_loaded--------")
    #门店编码转码
    #store_location['storeCode'] =store_location['storeCode'].astype(str)
    #merge两张表
    #temp_store_with_location = temp_province_store.merge(store_location,how='left',left_on='门店编码',right_on='storeCode')
    #calculate density for each region
    for county in temp_store_with_location['countyId'].unique():
        try:
            int(county)
        except:
            #排除没有经纬度信息的门店
            no_county_store = temp_store_with_location[temp_store_with_location['countyId'].isnull()].copy()
            no_county_store['lt20'] = 0
            no_county_store['gt20_lt60'] = 0
            no_county_store['gt60'] = 0
            province_list.append(no_county_store)
            continue
        #调取所有关店区域正常营业的门店
        temp_county_store_2_close = temp_store_with_location[temp_store_with_location['countyId'] == county].copy()
        temp_county_store_all = pd.read_sql("select storeCode,countyId,mapX,mapY from frxs_base.t_store where storeStatus = 'NORMAL' and mapX >0 and mapY >0 and countyId ="+str(county),conn)
         
        temp_county_store_all['flat_x'],temp_county_store_all['flat_y'] = zip(*temp_county_store_all.apply(lambda df: p(df['mapX'], df['mapY']), axis=1))     
        temp_county_store_all['point_xy'] =temp_county_store_all.apply(lambda df:Point(df['flat_x'],df['flat_y']),axis=1) 
        #create a buffer for all store
        temp_county_store_all['buffer'] = temp_county_store_all.apply(lambda df:df['point_xy'].buffer(500, cap_style=1),axis=1)
        #算各区域的intersection
        temp_county_store_2_close['lt20'],temp_county_store_2_close['gt20_lt60'],temp_county_store_2_close['gt60'] = zip(*temp_county_store_2_close.apply(lambda df:form_intersection(df['门店编码'],df['buffer'],temp_county_store_all),axis=1))
        province_list.append(temp_county_store_2_close)
    province_df = pd.concat(province_list)
    province_df.to_excel('0125/'+province+'.xlsx')


湖北区域--------------
湖南区域--------------
江西区域--------------
河南区域--------------
四川区域--------------
重庆区域--------------
广东区域--------------
贵州区域--------------
广西区域--------------
山东区域--------------
河北区域--------------
陕西区域--------------
福建区域--------------


In [5]:
def form_intersection(store_id,buffer,county_store):
    try:
        if (buffer == 'POLYGON EMPTY'):
            return 0,0,0
    except:
        return 0,0,0
    temp_store = county_store[county_store['storeCode'] != str(store_id)].copy()
    temp_store['intersection'] = temp_store.apply(lambda df: buffer.intersection(df['buffer']),axis=1)
#     temp_store['intersection_ratio_200'] = temp_store.apply(lambda df:df['intersection'].area / (3.14*(200**2)),axis = 1)
    temp_store['intersection_ratio_500'] = temp_store.apply(lambda df:df['intersection'].area / (3.14*(500**2)),axis=1)                                                             
    lt20 = temp_store[temp_store['intersection_ratio_500'] < 0.2].shape[0]
    gt60 = temp_store[temp_store['intersection_ratio_500'] > 0.6].shape[0]
    middle = temp_store.shape[0] - gt60 - lt20
    return lt20,middle,gt60   
    
    